In [1]:

import torch
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset, DataLoader
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup

from src.utils.hf_utils import download_weights
from src.utils.vit_config import inizialize_model
from src.data.dataset import PatchFromH5Dataset
from src.rl.train import Trainer, TrainingArguments


/home/vcivale/UNI_UCB2/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
NUM_STEPS = 10000 
LEARNING_RATE = 3e-2
WEIGHT_DECAY = 0.0
DECAY_TYPE = "cosine"
WARMUP_STEPS = 500
IMG_SIZE = 224 
TRAIN_BATCH_SIZE = 8
VAL_BATCH_SIZE = 8
NUM_CLASSES = 6

In [3]:
HF_WEIGHTS_PATH = "/equilibrium/datasets/TCGA-histological-data/vit_weights_cache"
weights_path = download_weights(HF_WEIGHTS_PATH)

timm_pretrained_state_dict = torch.load(weights_path, map_location="cpu")

In [4]:
model = inizialize_model(timm_pretrained_state_dict, num_classes=NUM_CLASSES)

Model weights loaded successfully.


In [5]:
dataset = PatchFromH5Dataset(
    h5_dir='/equilibrium/datasets/TCGA-histological-data/hest_dataset/datasets--MahmoodLab--hest/snapshots/cf37675c2006e6dfcdaa084ddeca863d21a8ddbb/patches',
    transform=transforms.Compose([
        transforms.Resize(IMG_SIZE),
        transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),])
)

In [6]:
labels = [dataset.label_to_idx[dataset.sample_to_label[file.replace('.h5','')]]
          for (file, _) in dataset.data_index]

indices = list(range(len(dataset)))

# Split stratificato
train_idx, val_idx = train_test_split(
    indices,
    test_size=0.2,
    random_state=42,
    stratify=labels
)

# Crea i Subset
train_dataset = Subset(dataset, train_idx)
val_dataset = Subset(dataset, val_idx)

train_loader = DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=True, num_workers=16)
val_loader = DataLoader(val_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=False, num_workers=16)

In [7]:
loss_function = torch.nn.CrossEntropyLoss()

In [8]:
optimizer = torch.optim.SGD(
    model.parameters(),
    lr=LEARNING_RATE,
    momentum=0.9,
    weight_decay=WEIGHT_DECAY,
)

In [9]:
if DECAY_TYPE == "cosine":
    scheduler = get_cosine_schedule_with_warmup(
        optimizer,
        num_warmup_steps=WARMUP_STEPS,
        num_training_steps=NUM_STEPS
    )
else: # DECAY_TYPE == "linear"
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=WARMUP_STEPS,
        num_training_steps=NUM_STEPS
    )

In [10]:
training_params = {
    "name": "vit_training_notebook_run",
    "output_dir": "./output_notebook",
    "eval_every": 50,
    "num_steps": NUM_STEPS,
    "learning_rate": LEARNING_RATE, # Passa l'LR se il trainer lo calcola internamente
    "weight_decay": WEIGHT_DECAY,
    "decay_type": DECAY_TYPE,
    "warmup_steps": WARMUP_STEPS,
    "max_grad_norm": 1.0,
    "local_rank": -1, # Usa -1 per non distribuito in un singolo notebook
    "seed": 42,
    "gradient_accumulation_steps": 1,
    "fp16": True, # Abilita o disabilita AMP
    "img_size": IMG_SIZE, # Necessario per UCB_Count_Score
    "train_batch_size": TRAIN_BATCH_SIZE, # Necessario per UCB_Count_Score
     "num_classes": NUM_CLASSES,
}

args = TrainingArguments(**training_params)

trainer = Trainer(
    args=args,
    model=model,
    train_dataloader=train_loader,
    eval_dataloader=val_loader,
    loss_function=loss_function,
    optimizer=optimizer, 
    scheduler=scheduler 
)

Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: True


wandb: Currently logged in as: vincenzo-civale (vincenzo-civale-universi-degli-studi-di-firenze) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/home/vcivale/UNI_UCB2/src/rl/train.py:122: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = GradScaler() if self.args.fp16 else None
07/14/2025 13:23:45 - INFO - src.rl.train - Training arguments: TrainingArguments(
    name = vit_training_notebook_run,
    output_dir = ./output_notebook,
    eval_every = 50,
    learning_rate = 0.03,
    weight_decay = 0.0,
    num_steps = 10000,
    decay_type = cosine,
    warmup_steps = 500,
    max_grad_norm = 1.0,
    local_rank = -1,
    seed = 42,
    gradient_accumulation_steps = 1,
    fp16 = True,
    num_classes = 6,
    img_size = 224,
    train_batch_size = 8,
    n_gpu = 1,
    device = cuda:0,
)
07/14/2025 13:23:45 - INFO - src.rl.train - Total parameters: 303.3M


In [ ]:
trainer.train()

07/14/2025 13:23:45 - INFO - src.rl.train - ***** Running training *****
07/14/2025 13:23:45 - INFO - src.rl.train -   Total optimization steps = 10000
07/14/2025 13:23:45 - INFO - src.rl.train -   Instantaneous batch size per GPU = 8
07/14/2025 13:23:45 - INFO - src.rl.train -   Total train batch size (w. parallel, distributed & accumulation) = 8
07/14/2025 13:23:45 - INFO - src.rl.train -   Gradient Accumulation steps = 1
Epoch 1 Training (0 / 10000 Steps) (loss=X.X):   0%|| 0/2134 [00:00<?, ?it/s]/home/vcivale/UNI_UCB2/src/rl/train.py:310: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast() if self.args.fp16 else torch.no_grad():
/home/vcivale/UNI_UCB2/.venv/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:182: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `l